# 3. Data Wrangling

This section will be dedicated to isolating out important features and structuring them for use during modelling. <br> Categorical columns will need to be encoded and rows that will not add value to the accuracy will further be removed.

In [1]:
import pandas as pd
import gzip
import json
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

## 3.1 Data Import

In [2]:
#defining methods to import data

""" unzips gz and load to json"""
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)


""" use parse method to read and translate to pandas dataframe """
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [9]:
re_MS = pd.read_csv('data/re_MS.csv',na_filter=False)

In [10]:
re_DM = pd.read_csv('data/re_DM.csv',na_filter=False)

In [11]:
re_KS = pd.read_csv('data/re_KS.csv',na_filter=False)

In [12]:
re_MTV = pd.read_csv('data/re_MTV.csv',na_filter=False)

In [3]:
pr_MS = pd.read_csv('data/pr_MS.csv',na_filter=False)

In [4]:
pr_DM = pd.read_csv('data/pr_DM.csv',na_filter=False)

In [5]:
pr_KS = pd.read_csv('data/pr_KS.csv',na_filter=False)

In [6]:
pr_MTV = pd.read_csv('data/pr_MTV.csv',na_filter=False)

## 3.2 Low Review Count Trimming

One of the goals of this project is to determine the overall popularity of the product based on various reviews provided from the customer. <br> However, if there are products with a single or just a few reviews, it may not represent well what the general population would feel about such products. <br> Because of this reason, any products with 4 or less reviews will be removed.

In [13]:
'''
Takes review dataset as input

Prints asin data characteristics:  
number of unique asin in dataset,  number of asin with a single review,
number of asin with two to four reviews, number of reviews with five or more reviews.

'''

def analyzeASIN(df):
    
    
    #check how many unique asin are present

    print('number of unique asin: ',len(df.asin.unique()))
    
    #create a groupby pandas series to isolate asin and their counts
    
    asin_count = df.groupby('asin').asin.count()
    
    #prints number of single reviewed asin
    
    print('number of single reviewed asin: ', asin_count[asin_count==1].sum())
    
    #print number of asin having between two to four reviews
    
    print('number of 2 reviewed asin: ', asin_count[asin_count==2].count())
    
    #print number of asin having between two to four reviews
    
    print('number of 3 reviewed asin: ', asin_count[asin_count==3].count())
    
    #print number of asin having between two to four reviews
    
    print('number of 4 reviewed asin: ', asin_count[asin_count==4].count())
    
    #print number of asin having five or more reviews
    
    print('number of 5 or more reviewed asin: ', asin_count[asin_count>=5].count())


### 3.2.1 Dataset 're_MS'

In [14]:
re_MS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88494 entries, 0 to 88493
Data columns (total 6 columns):
overall       88494 non-null float64
verified      88494 non-null bool
reviewerID    88494 non-null object
asin          88494 non-null object
reviewText    88494 non-null object
summary       88494 non-null object
dtypes: bool(1), float64(1), object(4)
memory usage: 3.5+ MB


In [15]:
analyzeASIN(re_MS)

number of unique asin:  2428
number of single reviewed asin:  413
number of 2 reviewed asin:  246
number of 3 reviewed asin:  176
number of 4 reviewed asin:  146
number of 5 or more reviewed asin:  1447


Isolating the reviews with 5 or more reduces the overall dataset to about 60% of the original. <br> It is important to save the list of asin with 4 or less reviews as they will not be considered in the product dataset as well.  <br> Moreover, the product dataset may include asin that did not have any reviews as well.  <br> Therefore, we will keep both asin list of 4 or less, and 5 or more.

In [16]:
#group by asin with count
asin_count = re_MS.groupby('asin').asin.count()

#less_* : contains asin of 4 or less
#more_* : contains asin of 5 or more

less_MS = asin_count[asin_count<5].index
more_MS = asin_count[asin_count>4].index

#remove from review dataset any asin with less than 5 reviews.
re_MS = re_MS.loc[re_MS.asin.isin(more_MS)]

print('expected dataset count after trimming: ',asin_count[asin_count>4].sum())

expected dataset count after trimming:  86477


In [17]:
re_MS.shape

(86477, 6)

### 3.2.2 Dataset 're_DM'

In [18]:
re_DM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1519311 entries, 0 to 1519310
Data columns (total 6 columns):
overall       1519311 non-null float64
verified      1519311 non-null bool
reviewerID    1519311 non-null object
asin          1519311 non-null object
reviewText    1519311 non-null object
summary       1519311 non-null object
dtypes: bool(1), float64(1), object(4)
memory usage: 59.4+ MB


In [19]:
analyzeASIN(re_DM)

number of unique asin:  456992
number of single reviewed asin:  290397
number of 2 reviewed asin:  70959
number of 3 reviewed asin:  29527
number of 4 reviewed asin:  16045
number of 5 or more reviewed asin:  50064


Around half of the digital music available in the dataset, over half of them contains only 1 reviews. <br> Keeping asin with 5 or more reviews will lower the data count to almost 10% of the original.

In [20]:
#group by asin with count
asin_count = re_DM.groupby('asin').asin.count()

#less_* : contains asin of 4 or less
#more_* : contains asin of 5 or more

less_DM = asin_count[asin_count<5].index
more_DM = asin_count[asin_count>4].index

#remove from review dataset any asin with less than 5 reviews.
re_DM = re_DM.loc[re_DM.asin.isin(more_DM)]

print('expected dataset count after trimming: ',asin_count[asin_count>4].sum())

expected dataset count after trimming:  934235


In [21]:
re_DM.shape

(934235, 6)

### 3.2.3 Dataset 're_KS'

In [22]:
re_KS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5709316 entries, 0 to 5709315
Data columns (total 6 columns):
overall       float64
verified      bool
reviewerID    object
asin          object
reviewText    object
summary       object
dtypes: bool(1), float64(1), object(4)
memory usage: 223.2+ MB


In [23]:
analyzeASIN(re_KS)

number of unique asin:  493849
number of single reviewed asin:  146741
number of 2 reviewed asin:  80060
number of 3 reviewed asin:  49127
number of 4 reviewed asin:  32570
number of 5 or more reviewed asin:  185351


In [24]:
#group by asin with count
asin_count = re_KS.groupby('asin').asin.count()

#less_* : contains asin of 4 or less
#more_* : contains asin of 5 or more

less_KS = asin_count[asin_count<5].index
more_KS = asin_count[asin_count>4].index

#remove from review dataset any asin with less than 5 reviews.
re_KS = re_KS.loc[re_KS.asin.isin(more_KS)]

print('expected dataset count after trimming: ',asin_count[asin_count>4].sum())

expected dataset count after trimming:  5124794


In [25]:
re_KS.shape

(5124794, 6)

### 3.2.4 Dataset 're_MTV'

In [26]:
re_MTV.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8526920 entries, 0 to 8526919
Data columns (total 6 columns):
overall       8526920 non-null float64
verified      8526920 non-null bool
reviewerID    8526920 non-null object
asin          8526920 non-null object
reviewText    8526920 non-null object
summary       8526920 non-null object
dtypes: bool(1), float64(1), object(4)
memory usage: 333.4+ MB


In [27]:
analyzeASIN(re_MTV)

number of unique asin:  182032
number of single reviewed asin:  44160
number of 2 reviewed asin:  23357
number of 3 reviewed asin:  14617
number of 4 reviewed asin:  10358
number of 5 or more reviewed asin:  89540


In [28]:
#group by asin with count
asin_count = re_MTV.groupby('asin').asin.count()

#less_* : contains asin of 4 or less
#more_* : contains asin of 5 or more

less_MTV = asin_count[asin_count<5].index
more_MTV = asin_count[asin_count>4].index

#remove from review dataset any asin with less than 5 reviews.
re_MTV = re_MTV.loc[re_MTV.asin.isin(more_MTV)]

print('expected dataset count after trimming: ',asin_count[asin_count>4].sum())

expected dataset count after trimming:  8350763


In [29]:
re_MTV.shape

(8350763, 6)

## 3.3 Dataset 'product' catalogue ASIN Trimming

As mentioned in the above section, we will now use the ASIN lists we saved during distinguishing ASIN with 4 or less review, and 5 or more reviews. <br> These lists will be used to trim out products that either have too few reviews to be viable, or have no reviews at all.

### 3.3.1 Dataset 'pr_MS'

In [30]:
pr_MS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2428 entries, 0 to 2427
Data columns (total 3 columns):
main_cat    2428 non-null object
asin        2428 non-null object
title       2428 non-null object
dtypes: object(3)
memory usage: 57.0+ KB


In [31]:
#trim down to only contain asin that has 5 or more reviews
pr_MS = pr_MS.loc[pr_MS.asin.isin(more_MS)]

In [32]:
pr_MS.shape

(1447, 3)

### 3.3.2 Dataset 'pr_DM'

In [33]:
pr_DM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457010 entries, 0 to 457009
Data columns (total 2 columns):
title    457010 non-null object
asin     457010 non-null object
dtypes: object(2)
memory usage: 7.0+ MB


In [34]:
#trim down to only contain asin that has 5 or more reviews
pr_DM  = pr_DM.loc[pr_DM.asin.isin(more_DM)]

In [35]:
pr_DM.shape

(50064, 2)

### 3.3.3 Dataset 'pr_KS'

In [36]:
pr_KS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493552 entries, 0 to 493551
Data columns (total 3 columns):
title       493552 non-null object
main_cat    493552 non-null object
asin        493552 non-null object
dtypes: object(3)
memory usage: 11.3+ MB


In [37]:
#trim down to only contain asin that has 5 or more reviews
pr_KS  = pr_KS.loc[pr_KS.asin.isin(more_KS)]

In [38]:
pr_KS.shape

(185141, 3)

### 3.3.4 Dataset 'pr_MTV'

In [39]:
pr_MTV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182043 entries, 0 to 182042
Data columns (total 3 columns):
title       182043 non-null object
main_cat    182043 non-null object
asin        182043 non-null object
dtypes: object(3)
memory usage: 4.2+ MB


In [40]:
#trim down to only contain asin that has 5 or more reviews
pr_MTV  = pr_MTV.loc[pr_MTV.asin.isin(more_MTV)]

In [41]:
pr_MTV.shape

(89540, 3)

In [42]:
re_MS.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/re_MS_5.csv',index=False)
re_DM.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/re_DM_5.csv',index=False)
re_KS.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/re_KS_5.csv',index=False)
re_MTV.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/re_MTV_5.csv',index=False)

In [43]:
del re_MS,re_DM,re_KS,re_MTV

In [44]:
pr_MS.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/pr_MS_5.csv',index=False)
pr_DM.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/pr_DM_5.csv',index=False)
pr_KS.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/pr_KS_5.csv',index=False)
pr_MTV.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone2/data/pr_MTV_5.csv',index=False)

In [45]:
del pr_MS,pr_DM,pr_KS,pr_MTV